In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def process_2021_month(file, year, month):
    # # Table 11a 시트(england) / # Table 11b 시트(wales) 불러오기 
    england_df = pd.read_excel(file, sheet_name='Table 11a') 
    wales_df = pd.read_excel(file, sheet_name='Table 11b')
    # 필요한 부분만 사용 
    england_df = england_df.iloc[5:15, :4] 
    wales_df = wales_df.iloc[5:15, :4]
    # 컬럼 이름 재정의 -> [질병, 순위, 사망자 수, 100000명당 사망자 수] 
    england_df.columns = ['disease', 'rank', 'num_deaths', 'deaths_per_100000'] 
    wales_df.columns = ['disease', 'rank', 'num_deaths', 'deaths_per_100000']
    # rank열 필요없어서 drop 
    england_df = england_df.drop(['rank'], axis=1)
    wales_df = wales_df.drop(['rank'], axis=1)
    # country(나라) 컬럼 생성 
    england_df['country'] = 'England'
    wales_df['country'] = 'Wales'
    # 두개의 df concat
    merged = pd.concat([england_df, wales_df])
    # date 컬럼 생성 
    merged['date'] = f"{year}-{month}"
    return merged
    

In [ ]:
def process_2021():
    to_return_df = pd.DataFrame()
    for month in range(1, 13): # month = 1~12까지 반복 
        file_name = f"uk_data/2021_{month:02}_monthly_mortality.xlsx" 
        to_return_df = pd.concat([to_return_df, process_2021_month(file_name, 2021, month)])
    return to_return_df

In [ ]:
def process_2022_month(file, year, month):  # 2022-01~10월 concat하는 함수 
    start_row = 6
    end_row = 16
    if month > 5:
        start_row = 7
        end_row = 17
    england_df = pd.read_excel(file, sheet_name='11a')
    wales_df = pd.read_excel(file, sheet_name='11b')
    england_df = england_df.iloc[start_row:end_row, :4]
    wales_df = wales_df.iloc[start_row:end_row, :4]
    england_df.columns = ['disease', 'rank', 'num_deaths', 'deaths_per_100000']
    wales_df.columns = ['disease', 'rank', 'num_deaths', 'deaths_per_100000']
    england_df = england_df.drop(['rank'], axis=1)
    wales_df = wales_df.drop(['rank'], axis=1)
    england_df['country'] = 'England'
    wales_df['country'] = 'Wales'
    merged = pd.concat([england_df, wales_df])
    merged['date'] = f"{year}-{month}"
    
    return merged

In [ ]:
def process_2022():
    to_return_df = pd.DataFrame()
    for month in range(1, 11):  # month = 1~10까지 반복 
        file_name = f"uk_data/2022_{month:02}_monthly_mortality.xlsx"
        to_return_df = pd.concat([to_return_df, process_2022_month(file_name, 2022, month)])
    return to_return_df

In [ ]:
def process_2022_11(): # 2022-11 concat df
    year=2022
    month=11
    file= f"uk_data/2022_11_monthly_mortality.xlsx"
    england_df = pd.read_excel(file, sheet_name='12a')
    wales_df = pd.read_excel(file, sheet_name='12b')
    england_df = england_df.iloc[7:17, :4]
    wales_df = wales_df.iloc[7:17, :4]
    england_df.columns = ['disease', 'rank', 'num_deaths', 'deaths_per_100000']
    wales_df.columns = ['disease', 'rank', 'num_deaths', 'deaths_per_100000']
    england_df = england_df.drop(['rank'], axis=1)
    wales_df = wales_df.drop(['rank'], axis=1)
    england_df['country'] = 'England'
    wales_df['country'] = 'Wales'
    merged = pd.concat([england_df, wales_df])
    merged['date'] = f"{year}-{month}"
    return merged

In [ ]:
def process_2022_12(): # 2022-12 concat df
    year=2022
    month=12
    file= f"uk_data/2022_12_monthly_mortality.xlsx"
    england_df = pd.read_excel(file, sheet_name='12a')
    wales_df = pd.read_excel(file, sheet_name='13a')
    england_df = england_df.iloc[6:16, :4]
    wales_df = wales_df.iloc[6:16, :4]
    england_df.columns = ['rank', 'disease',  'num_deaths', 'deaths_per_100000']
    wales_df.columns = ['rank', 'disease', 'num_deaths', 'deaths_per_100000']
    england_df = england_df.drop(['rank'], axis=1)
    wales_df = wales_df.drop(['rank'], axis=1)
    england_df['country'] = 'England'
    wales_df['country'] = 'Wales'
    merged = pd.concat([england_df, wales_df])
    merged['date'] = f"{year}-{month}"
    return merged

In [ ]:
def process_all():  # 위의 함수를 모두 실행
    to_return_df = process_2021()
    to_return_df = pd.concat([to_return_df, process_2022()])
    to_return_df = pd.concat([to_return_df, process_2022_11()])
    to_return_df = pd.concat([to_return_df, process_2022_12()])
    return to_return_df

In [ ]:
def clean_disease(df): # disease 컬럼 전처리 
    df['disease'] = df['disease'].apply(lambda x: x.replace("'s", ""))
    df['disease'] = df['disease'].apply(lambda x: x.replace(",", ""))
    df['disease'] = df['disease'].apply(lambda x: x.strip())
    df['disease'] = df['disease'].apply(lambda x: x.lower())
    df['disease'] = df['disease'].apply(lambda x: x.replace('malignant neoplasms stated or presumed to be primary of lymphoid haematopoietic and related tissue', 'malignant neoplasms of lymphoid haematopoietic and related tissue'))
    df['disease'] = df['disease'].apply(lambda x: x.replace('malignant neoplasms of breast', 'malignant neoplasm of breast'))
    df = df[ df['disease'] != 'covid-19']
    return df

In [ ]:
def add_weather_columns(df): # weather_data 전처리
    month_int_map = {
    "JAN": 1,
    "FEB": 2,
    "MAR": 3,
    "APR": 4,
    "MAY": 5,
    "JUN": 6,
    "JUL": 7,
    "AUG": 8,
    "SEP": 9,
    "OCT": 10,
    "NOV": 11,
    "DEC": 12,
    }
    weather_df = pd.read_csv('uk_data/uk_weather_data.csv')
    weather_df = weather_df[ weather_df['ReportingPeriodType'] == 'Month']
    weather_df = weather_df[['Year', 'Period', 'Rainfall_mm', 'Temperature_C']]
    years =[2022, 2021]
    weather_df = weather_df[ weather_df['Year'].isin(years) ]
    weather_df['Period'] = weather_df['Period'].apply(lambda x: month_int_map[x])
    weather_df['date'] = weather_df['Year'].astype(str) + '-' + weather_df['Period'].astype(str)
    weather_df=weather_df.drop('Period', axis=1)
    weather_df=weather_df.drop('Year', axis=1)
    weather_df.columns = ['rainfall', 'temperature', 'date']
    merged_df = pd.merge(df, weather_df, on='date', how='left')
    merged_df['date'] = pd.to_datetime(merged_df['date'])
    merged_df['date'] = merged_df['date'].dt.to_period('M')
    merged_df['year'] = merged_df['date'].dt.year
    merged_df['month'] = merged_df['date'].dt.month
    return merged_df
    

In [ ]:
df = clean_disease(process_all())
df = add_weather_columns(df)

In [ ]:
to_remove = ['cardiac arrhythmias', 'cirrhosis and other diseases of liver', 'diseases of the urinary system', 'heart failure and complications and ill-defined heart disease', 'malignant neoplasm of breast', 'malignant neoplasm of pancreas', 'malignant neoplasm of prostate']
df = df[ ~df['disease'].isin(to_remove) ]

In [ ]:
df

,disease,num_deaths,deaths_per_100000,country,date,rainfall,temperature,year,month
0,dementia and alzheimer disease,5655,119.4,England,2021-01,132.0,1.9,2021,1
1,ischaemic heart diseases,4833,103,England,2021-01,132.0,1.9,2021,1
2,cerebrovascular diseases,2516,53.4,England,2021-01,132.0,1.9,2021,1
3,malignant neoplasm of trachea bronchus and lung,2274,48.6,England,2021-01,132.0,1.9,2021,1
4,chronic lower respiratory diseases,2025,43.1,England,2021-01,132.0,1.9,2021,1
...,...,...,...,...,...,...,...,...,...
435,influenza and pneumonia,206,70.3,Wales,2022-12,80.4,2.9,2022,12
436,cerebrovascular diseases,174,58,Wales,2022-12,80.4,2.9,2022,12
437,malignant neoplasm of trachea bronchus and lung,140,45.8,Wales,2022-12,80.4,2.9,2022,12
438,symptoms signs and ill-defined conditions,127,45.1,Wales,2022-12,80.4,2.9,2022,12


In [ ]:
#df.to_csv("combined_mortality_and_weather_2021_2022.csv",index=False) # 전처리 완료 데이터 저장 

In [ ]:
df['disease'].unique() # 질병 종류 9가지 

array(['dementia and alzheimer disease', 'ischaemic heart diseases',
       'cerebrovascular diseases',
       'malignant neoplasm of trachea bronchus and lung',
       'chronic lower respiratory diseases',
       'symptoms signs and ill-defined conditions',
       'influenza and pneumonia',
       'malignant neoplasm of colon sigmoid rectum and anus',
       'malignant neoplasms of lymphoid haematopoietic and related tissue'],
      dtype=object)

In [ ]:
# dementia and alzheimer disease(치매 및 알츠하이머) / 48
# ischaemic heart diseases(허혈성 심장질환) / 48
# cerebrovascular diseases(뇌혈관질환) / 48
# malignant neoplasm of trachea bronchus and lung(악성 기관, 기관지 및 폐 신생물 = 폐암) / 48
# chronic lower respiratory diseases(만성폐쇄성폐질환, 천식, 폐섬유증) / 48
# influenza and pneumonia(인플루엔자 및 폐렴) / 48
# malignant neoplasm of colon sigmoid rectum and anus(대장암 및 직장암) / 48
# symptoms signs and ill-defined conditions(의학적으로 명확게 진단되지 않은 상태_주요증상:발열,통증,피로,어지러움,불명확한 복통) / 45
# malignant neoplasms of lymphoid haematopoietic and related tissue(혈액암_림프종,백혈병,다발성 골수증) / 33 

In [ ]:
# 각 질병별 온도와 사망률 간의 상관관계 
disease_corr = df.groupby('disease').apply(lambda x: x[['temperature', 'deaths_per_100000']].corr().iloc[0, 1])
print(disease_corr.sort_values())

disease
dementia and alzheimer disease                                      -0.568921
ischaemic heart diseases                                            -0.535644
influenza and pneumonia                                             -0.480381
cerebrovascular diseases                                            -0.456367
symptoms signs and ill-defined conditions                           -0.370868
chronic lower respiratory diseases                                  -0.282930
malignant neoplasm of colon sigmoid rectum and anus                 -0.018070
malignant neoplasms of lymphoid haematopoietic and related tissue    0.064148
malignant neoplasm of trachea bronchus and lung                      0.065348
dtype: float64


C:\Users\Jo\AppData\Local\Temp\ipykernel_22580\1140365941.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  disease_corr = df.groupby('disease').apply(lambda x: x[['temperature', 'deaths_per_100000']].corr().iloc[0, 1])


In [ ]:
# 각 질병별 강수량과 사망률 간의 상관관계 
disease_corr = df.groupby('disease').apply(lambda x: x[['rainfall', 'deaths_per_100000']].corr().iloc[0, 1])
disease_corr

C:\Users\Jo\AppData\Local\Temp\ipykernel_22580\13484962.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  disease_corr = df.groupby('disease').apply(lambda x: x[['rainfall', 'deaths_per_100000']].corr().iloc[0, 1])


disease
cerebrovascular diseases                                             0.313551
chronic lower respiratory diseases                                   0.122804
dementia and alzheimer disease                                       0.228459
influenza and pneumonia                                              0.198849
ischaemic heart diseases                                             0.092428
malignant neoplasm of colon sigmoid rectum and anus                 -0.092104
malignant neoplasm of trachea bronchus and lung                     -0.227543
malignant neoplasms of lymphoid haematopoietic and related tissue   -0.099958
symptoms signs and ill-defined conditions                            0.301459
dtype: float64